In [14]:
# !pip install -U datasets

In [8]:
# !sudo -H pip install -q transformers --upgrade

In [15]:
# !pip uninstall autoawq -y

In [16]:
# !pip install autoawq

In [17]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [18]:
# !pip install transformers==4.35.2

In [1]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM

In [3]:
import torch
torch.__version__

'2.0.1+cu117'

In [8]:
#model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [4]:
quant_path = '/llmmodels/quantized_model/mistral'

In [5]:
model_path='mistralai/Mistral-7B-v0.1'

In [6]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [7]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [9]:
import torch
torch.cuda.empty_cache()

In [10]:
import gc
gc.collect()


31

In [11]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ:  22%|██▏       | 7/32 [08:58<32:22, 77.71s/it]

In [11]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [16]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

/tmp/pip_packages/transformers/generation/configuration_utils.py:527: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


('/llmmodels/quantized_model/awq/tokenizer_config.json',
 '/llmmodels/quantized_model/awq/special_tokens_map.json',
 '/llmmodels/quantized_model/awq/tokenizer.model',
 '/llmmodels/quantized_model/awq/added_tokens.json',
 '/llmmodels/quantized_model/awq/tokenizer.json')

### Inference on quantized model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [14]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [15]:
import time

In [23]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 21.147445678710938


In [24]:
print(output1)

</s>what is art?

Art is a way of expressing one's self. It is a way of expressing one's thoughts, feelings, and ideas. It is a way of expressing one's self through the use of various media.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art for

### Inference on un-quantized model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
model_path = '/data/quantization-trials/merged-model'

In [20]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

time taken is : 23.784337282180786


In [26]:
print(output2)

</s>what is art?

Art is a creative expression of the human mind. It is a way of expressing the human mind through the use of various media.

The word art is derived from the Latin word artus, which means "to make". The word art is used in many different contexts, including:

- to describe the beauty of a work of art

- to describe the meaning of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work 

In [26]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [27]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out